In [41]:
print("Trial graph for all the agents")

Trial graph for all the agents


IMPORTING ALL THE REQUIRED PACKAGES

In [42]:
!pip install -qU google-genai langchain openai python-dotenv beautifulsoup4 matplotlib networkx

In [43]:
import research_agent  , question_generator_agent , coding_agent , research_2

In [4]:
from typing import TypedDict, List
import matplotlib.pyplot as plt

CREATING THE GRAPH STATE ---> this is a shared data source in the entire graph

In [44]:
class State(TypedDict):
    topic:str # this is the main concept/algorithm/idea to research and frame questions on 
    research_json: dict # this will hold the json output from the research agent
    question: dict # this is formatted data of the question that is generated by the question generator agent
    solution: dict # this field is the answer to the problem , need to be filled optionally
    human_feedback: str # this field is ment to add human feedback , mostly regarding to the question 
    user_comments: str # this field is ment to add human feedback , mostly regarding to the solution

Creating a simple researcher node

In [45]:
from research_2 import JsonOutput , ResearchAgent


def research_node(state : State):
    """ this is a node for the researcher agent , to perform web search and curate
    information on the requested topic"""

    
    print("********* Researcher node is running**********")
    topic = state['topic']

    # create the research object and call the functions
    research_agent = ResearchAgent()
    output = research_agent.search(topic)
    print(f"the final output got from the research agent  is -->\n\n {output}" )

    # the final step is to add the data or any changes to the graph state
    return {"research_json": output}

Creating the question creator node

In [46]:
def question_crafter_node(state : State):
    """  This node is to be used for crafting the question from the research json object
    Here the key insights from the research will be transformed into a question after analyzing it and taking into consideration all the key technical details.
    """

    print("*********** Question Crafter node is running ****************")

    #1. take the research json object from the graph state
    research_json = state['research_json'] # this is the knowledge packet for creating the question

    #2. also check if there is any comments form the user regarding the question
    comments = state.get('user_comments' , "")
    prompt_enhancer = ""
    print(f"the rejection reason by the user is : \n\n {comments}\n\n Now modifying the prompt accordingly")

    if comments:
        prompt_enhancer = """The question you gave was rejected by the user , the reasons the user gave are these {comments} . 
           Please consider these points and improve the question quality accordingly .
           And one more important thing , DO NOT REGENERATE THE SAME QUESTION AGAIN . """.format(comments=comments)

        print(f" The modified prompt is {prompt_enhancer}")
    # serialize this data
    import json
    from question_generator import QuestionGenerator
    question_creator = QuestionGenerator()
    research_string_data = json.dumps(research_json)
    generated_question = question_creator.generate_question(research_string_data , prompt_enhancer=prompt_enhancer)
    print(f"\n The question generated by the agent is : \n\n {generated_question}")
    #update the graph state with this question
    return {"question": generated_question.model_dump() , "human_feedback": "" , "user_comments": ""}

Creating the coder agent 

In [47]:
def question_solver(state:State):
    """ This node is to be used for solving the question that is generated by the question generator agent
    This will involve analyzing the question and generating a solution based on the constraints given and generating the most optimal solution.
    """

    print("*********** Question Solver node is running ****************")

    #1. take the research json object from the graph state
    question = state['question'] 
    # this data needs to be converted into the pydantic model 
    from question_generator_agent import CodeQuestion # this is the data model
    import coding_agent
    formatted_question = CodeQuestion(**question)
    #now give this to the llm to solve
    solution = coding_agent.solve_question(formatted_question)
    print(f"\n The solution generated by the agent is : \n\n {solution}")
    
    return {"solution" : solution }

CREATING A HUMAN IN THE GRAPH NODE

In [48]:
# This node is ment to add human intput in the graph 

def human_review(state: State):
    """ this is to take teh human feed back on the question generated """


    print("*********** Human review node is running ****************")

    #1. print the current question 
    print(f"The current generated question is : \n\n {state['question']}")
    #2. take the human input
    feedback = input("Please provide your feedback on the question generated : ")
    comments = "" 
    if feedback.strip() == "no":
        # here take the specific comments from the user
        comments = input("Please provide your comments on the question generated and the improvements u want: ")
    
    return {
            "human_feedback": feedback.lower(),
            "user_comments": comments.lower()
        }

Creating a router node

In [49]:
def router(state: State):
    """ this is a router node to decide the flow of the graph based on the human feedback """
    
    print("*********** Router node is running ****************")

    feedback = state.get("human_feedback", "")
    
    # CHECK IF THE FEEDBACK IS POSITIVE OR NOT 
    if feedback == "yes":
        return "solver"
    elif feedback == "no":
        return "question_crafter"

CREATING THE GRAPH 

graph design 2

In [50]:
from langgraph.graph import START , END , StateGraph

# 1. Instantiate the graph
graph = StateGraph(State)

# 2. Add all the nodes
graph.add_node("Researcher", research_node)
graph.add_node("Question Generator", question_crafter_node)
graph.add_node("Human Review", human_review)
graph.add_node("Question Solver", question_solver)

# 3. Define the edges
graph.set_entry_point("Researcher")
graph.add_edge("Researcher", "Question Generator")
graph.add_edge("Question Generator", "Human Review")

# The conditional edge now only chooses between solving or regenerating
graph.add_conditional_edges(
    "Human Review",
    router,
    {
        "solver": "Question Solver",
        "question_crafter": "Question Generator"
    }
)

# The ONLY path to the end is from the Question Solver
graph.add_edge("Question Solver", END)

# 4. Compile the graph
complete_dsa_agent = graph.compile()

In [51]:
# printing the graph 

print(complete_dsa_agent.get_graph().print_ascii())

    +-----------+      
    | __start__ |      
    +-----------+      
           *           
           *           
           *           
    +------------+     
    | Researcher |     
    +------------+     
           *           
           *           
           *           
+--------------------+ 
| Question Generator | 
+--------------------+ 
           *           
           *           
           *           
   +--------------+    
   | Human Review |    
   +--------------+    
           .           
           .           
           .           
  +-----------------+  
  | Question Solver |  
  +-----------------+  
           *           
           *           
           *           
      +---------+      
      | __end__ |      
      +---------+      
None


TESTING THE GRAPH WORK FLOW

In [52]:
# --- FINAL TEST BLOCK ---
if __name__ == "__main__":
    import json

    # Define the initial state to kick off the graph
    initial_state = {
        "topic": "key concepts on binary search algorithm and most common questions on it"
    }

    print("🚀 Invoking the LangGraph Application...")
    
    # The .invoke() method runs the graph from the entry point to the end.
    final_state = complete_dsa_agent.invoke(initial_state)

    print("\n--- ✅ Graph Execution Complete ---")
    
    # The final state contains all the accumulated data.
    # We can now access the generated question and its solution.
    
    print("\n\n--- Generated Question ---")
    print(json.dumps(final_state['question'], indent=2))
    
    print("\n\n--- Generated Solution ---")
    print(json.dumps(final_state['solution'], indent=2))

🚀 Invoking the LangGraph Application...
********* Researcher node is running**********
Output of gemini api is : 
['The article "Top 10 Most Asked Binary Search Interview Questions" by Kirti Arora outlines essential binary search problems frequently encountered in technical interviews, highlighting the algorithm\'s O(log n) time complexity. It covers a range of scenarios, from basic implementation to more complex adaptations.\n\nHere\'s a breakdown of the key questions and their technical/coding aspects:\n\n1.  **Basic Binary Search:** Implements the core iterative or recursive approach to find a target in a sorted array, moving `left` and `right` pointers based on comparison with the `mid` element.\n2.  **Find the First and Last Position of an Element in a Sorted Array:** Requires two modified binary searches; one to find the first occurrence by continuing to search left after finding the target, and another to find the last by continuing to search right.\n3.  **Search Insert Position